In [1]:
import cv2
import os

In [2]:

cap = cv2.VideoCapture(0)


text_file_path = "detected_letters.txt"

print("Press 'd' to detect and save the current letter, 'q' to quit.")

Press 'd' to detect and save the current letter, 'q' to quit.


In [3]:
def load_templates(directory="cropped_captured_templates"):
    templates = {}
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):  
            name = filename.split('.')[0]
            img_path = os.path.join(directory, filename)
            template = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            templates[name] = template
    return templates

templates = load_templates()

In [4]:
def find_best_match(frame, templates):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    best_score = 0
    best_name = None
    best_loc = None
    template_size = (0, 0) 

    for name, template in templates.items():
        res = cv2.matchTemplate(gray_frame, template, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        if max_val > best_score:
            best_score = max_val
            best_name = name
            best_loc = max_loc
            template_size = template.shape[::-1]  

    return best_name, best_score, best_loc, template_size

In [5]:
def calculate_orientation(contour):
    if len(contour) < 5:
        return 0
    (x, y), (MA, ma), angle = cv2.fitEllipse(contour)
    return angle

In [6]:
templates = load_templates()

text_file_path = "detected_letters.txt"
video_file_path = "detection_session.mp4"
is_recording = False
video_writer = None
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

In [13]:
cap = cv2.VideoCapture(0)
print("Press 'd' to save the detected letter, 'r' to start/stop recording, 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    best_match, score, loc, size = find_best_match(frame, templates)
    
    if score > 0.7:
        top_left = loc
        bottom_right = (top_left[0] + size[0], top_left[1] + size[1])
        cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
        cv2.putText(frame, f'Match: {best_match}, Score: {score:.2f}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    cv2.imshow("Real-time Template Matching", frame)
    
    key = cv2.waitKey(1) & 0xFF

    if key == ord('d') and score > 0.7:
        with open(text_file_path, "a") as file:
            file.write(best_match + "\n")
        print(f"Detected letter '{best_match}' saved.")

    elif key == ord('r'):
        if is_recording:
            is_recording = False
            video_writer.release()
            print("Stopped recording.")
        else:
            is_recording = True
            video_writer = cv2.VideoWriter(video_file_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))
            print("Started recording.")

    elif key == ord('q'):
        break

    if is_recording:
        video_writer.write(frame)

if is_recording:
    video_writer.release()
cap.release()
cv2.destroyAllWindows()

Press 'd' to save the detected letter, 'r' to start/stop recording, 'q' to quit.
Started recording.
Detected letter 'S' saved.
Detected letter 'A' saved.
Detected letter 'U' saved.
Detected letter 'N' saved.
Detected letter 'A' saved.
Detected letter 'K' saved.
Stopped recording.
